In [ ]:
import os
#GPU_NO = "3"
#os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NO
TRANSFORMERS_CACHE='./'
import json

In [ ]:
import sys
MGIT_PATH=os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(MGIT_PATH)
from utils.lineage.graph import *
from utils import meta_functions

In [ ]:
success_condition_path=os.getcwd()+'/user_functions.py'
success_condition_name='test_success_condition'

In [ ]:
compute_metrics_path=os.getcwd()+'/user_functions.py'
compute_metrics_name='compute_metrics'

In [ ]:
lineage_eval_dataset = LineageDataset("nyu-mll/glue", "sst2", split="validation", cache_dir=TRANSFORMERS_CACHE, feature_keys=['sentence'])

In [ ]:
#lineage_train_dataset = LineageDataset("glue", "sst2", split="train", cache_dir=TRANSFORMERS_CACHE, feature_keys=['premise','hypothesis'])
lineage_train_dataset = LineageDataset("nyu-mll/glue", "sst2", split="validation", cache_dir=TRANSFORMERS_CACHE, feature_keys=['sentence'])

In [ ]:
preprocess_file = os.path.join(MGIT_PATH,'utils/preprocess_utils.py')
preprocess_function = 'glue_preprocess_function'

In [ ]:
!rm -rf tmp_sst2_node1
!rm -rf tmp_sst2_node2
!rm -rf tmp_sst2_node3
!rm -rf parameter_store
!rm *.json

In [ ]:
!../../../mgit init

In [ ]:
lineage_train = LineageTrain(
    preprocess_function_path=preprocess_file,
    preprocess_function_name=preprocess_function,
    train_dataset=lineage_train_dataset,
    eval_dataset=lineage_eval_dataset,
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)
node1 = LineageNode(
    init_checkpoint='roberta-base',
    lineage_train=lineage_train,
    output_dir='tmp_sst2_node1',
)
node_config = node1.to_json()
with open("root.json", "w") as fp:
    json.dump(node_config, fp)

In [ ]:
!../../../mgit add -node_config ./root.json

In [ ]:
lineage_train = LineageTrain(
    preprocess_function_path=preprocess_file,
    preprocess_function_name=preprocess_function,
    train_dataset=lineage_train_dataset,
    eval_dataset=lineage_eval_dataset,
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)
node2 = LineageNode(
    init_checkpoint='roberta-base',
    model_init_function_path=success_condition_path,
    model_init_function_name='vanilla_finetune_init_function',
    lineage_train=lineage_train,
    output_dir='tmp_sst2_node2',
    is_delta=True,
)
node2_config = node2.to_json()
with open("node2.json", "w") as fp:
    json.dump(node2_config, fp)

In [ ]:
!../../../mgit add -node_config ./node2.json -e adapted versioned -parent tmp_sst2_node1

In [ ]:
lineage_train = LineageTrain(
    preprocess_function_path=preprocess_file,
    preprocess_function_name=preprocess_function,
    train_dataset=lineage_train_dataset,
    eval_dataset=lineage_eval_dataset,
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)
node3 = LineageNode(
    init_checkpoint='roberta-base',
    model_init_function_path=success_condition_path,
    model_init_function_name='vanilla_finetune_init_function',
    lineage_train=lineage_train,
    output_dir='tmp_sst2_node3',
    is_delta=True,
)
node3_config = node3.to_json()
with open("node3.json", "w") as fp:
    json.dump(node3_config, fp)

In [ ]:
!../../../mgit add -node_config ./node3.json -e adapted versioned -parent tmp_sst2_node2

In [ ]:
test1 = LineageTest(
        preprocess_function_path=preprocess_file,
        preprocess_function_name=preprocess_function,        
        eval_dataset=lineage_eval_dataset,
        test_success_condition_path=success_condition_path,
        test_success_condition_name=success_condition_name,
        compute_metrics_path=compute_metrics_path,
        compute_metrics_name=compute_metrics_name,
        metric_for_best_model='accuracy',
        name='test1',
)
test_config = test1.to_json()
with open("test.json", "w") as fp:
    json.dump(test_config, fp)

In [ ]:
!../../../mgit test -n tmp_sst2_node1 -test_config ./test.json
!../../../mgit test -n tmp_sst2_node2 -test_config ./test.json
!../../../mgit test -n tmp_sst2_node3 -test_config ./test.json

In [ ]:
!../../../mgit train -n tmp_sst2_node1

In [ ]:
!../../../mgit train -n tmp_sst2_node2

In [ ]:
!../../../mgit train -n tmp_sst2_node3

In [ ]:
!../../../mgit test -n tmp_sst2_node1 -t test1 -exe

In [ ]:
!../../../mgit test -n tmp_sst2_node2 -t test1 -exe

In [ ]:
!../../../mgit test -n tmp_sst2_node3 -t test1 -exe

In [ ]:
!../../../mgit display_test -metrics